# [백준/아기 상어](https://www.acmicpc.net/problem/16236)

## 풀이과정

### 첫번째 시도

풀이과정

In [513]:
def solution():
    import sys

    inputs = iter(sys.stdin.read().rstrip().split("\n"))
    n = int(next(inputs))
    sea = {
        (x, y): int(value)
        for y, input in enumerate(inputs)
        for x, value in enumerate(input.split())
    }
    secs = 0
    size = 2
    poss = set()
    fishes = {}
    toeat = set()
    position = False
    stomach = 0

    def side(x, y):
        return {(x, y + 1), (x, y - 1), (x + 1, y), (x - 1, y)}

    for (x, y), value in sea.items():
        if value == 0:
            poss.add((x, y))
        elif value < 9:
            fishes[x, y] = value
            if value < size:
                poss.add((x, y))
                toeat.add((x, y))
            elif value == size:
                poss.add((x, y))
        elif not position:
            position = (x, y)
            poss.add((x, y))
    while fishes and [fish for fish in fishes.values() if fish < size]:
        x, y = position
        visited = {position}
        tovisit = side(x, y) & poss
        for m in range(1, n * n):
            if not tovisit:
                return
            if catch := tovisit & toeat:
                sea[position] = 0
                secs += m
                position = min(catch, key=lambda x: x[::-1])
                toeat -= {position}
                fishes.pop(position)
                stomach += 1
                sea[position] = 9
                if stomach == size:
                    stomach = 0
                    size += 1
                    for fish, value in fishes.items():
                        if value < size:
                            poss.add((x, y))
                            toeat.add(fish)
                        elif value == size:
                            poss.add(fish)
                break
            visited |= tovisit
            tovisit = set().union(*(side(x, y) for x, y in tovisit)) & poss
    print("\n", secs, sep="")


solution()

### 두번째 시도

In [ ]:
def solution():
    import sys
    from collections import deque

    inputs = iter(sys.stdin.read().rstrip().split("\n"))
    n = int(next(inputs))
    sea = [list(map(int, input.split())) for input in inputs]
    secs = 0
    size = 2
    fishes = {}
    toeat = set()
    stomach = 0
    nn = n * n
    dxy = ((0, -1), (-1, 0), (1, 0), (0, 1))
    for y, line in enumerate(sea):
        for x, cell in enumerate(line):
            if 0 < cell < 9:
                fishes[x, y] = cell
                if cell < size:
                    toeat.add((x, y))
            elif cell == 9:
                position = (x, y)
                sea[y][x] = 0
    x, y = position
    curr = [[-1] * n for _ in range(n)]
    curr[y][x] = 0
    for (x, y), fish in fishes.items():
        if fish > size:
            curr[y][x] = nn
    new_q = {position}
    while toeat:
        queue = deque(new_q)
        while queue:
            x, y = queue.popleft()
            for dx, dy in dxy:
                nx, ny = x + dx, y + dy
                if 0 <= nx < n and 0 <= ny < n:
                    if curr[ny][nx] == -1:
                        new_q.add((nx, ny))
                        curr[ny][nx] = curr[y][x] + 1
        if able := toeat & new_q:
            position = x, y = min(able, key=lambda x: x[::-1])
            secs += curr[y][x]
            fishes.pop(position)
            toeat.remove(position)
            stomach += 1
            if stomach == size:
                stomach = 0
                size += 1
                toeat.update({fish for fish, value in fishes.items() if value < size})
            sea[y][x] = 0
            new_q = {position}
            curr = [[-1] * n for _ in range(n)]
            curr[y][x] = 0
            for (x, y), fish in fishes.items():
                if fish > size:
                    curr[y][x] = nn
        elif not new_q:
            break

    print(secs)


solution()

### 세번째 시도

In [28]:
def solution():
    import sys
    from collections import deque
    from bisect import insort

    inputs = iter(sys.stdin.read().rstrip().split("\n"))
    n = int(next(inputs))
    sea = [list(map(int, input.split())) for input in inputs]

    secs = 0
    size = 2
    fishes = {}
    stomach = 0
    nn = n * n

    for y, line in enumerate(sea):
        for x, cell in enumerate(line):
            if 0 < cell < 9:
                fishes[x, y] = cell
            elif cell == 9:
                start = (x, y)
                sea[y][x] = 0

    def bfs(x, y, size, nn=nn, fishes=fishes, sea=sea):
        dists = [[-1] * n for _ in range(n)]
        dists[y][x] = 0
        queue = deque([(x, y)])
        for (x, y), fish in fishes.items():
            if fish > size:
                dists[y][x] = nn
        dist = nn
        targets = deque([])
        while queue:
            x, y = queue.popleft()
            for dx, dy in ((1, 0), (-1, 0), (0, 1), (0, -1)):
                nx, ny = x + dx, y + dy
                next_dist = dists[y][x] + 1
                if 0 <= nx < n and 0 <= ny < n and dists[ny][nx] == -1:
                    if (cell := sea[ny][nx]) <= size:
                        dists[ny][nx] = next_dist
                    if 0 < cell < size:
                        if dist > dists[ny][nx]:
                            dist = dists[ny][nx]
                            targets = deque([(nx, ny)])
                        elif dist == dists[ny][nx]:
                            insort(targets, (nx, ny), key=lambda x: x[::-1])
                    if next_dist <= dist:
                        queue.append((nx, ny))
        return False if dist == nn else (*targets.popleft(), dist)

    x, y = start
    while fishes:
        target = bfs(x, y, size)
        if not target:
            break
        x, y, sec = target
        secs += sec
        fishes.pop((x, y))
        stomach += 1
        if stomach == size:
            stomach = 0
            size += 1
        sea[y][x] = 0

    print(secs)


solution()

## 해답

In [1]:
from collections import deque
from bisect import insort

In [91]:
def bfs(x, y, size, n, fishes, sea):
    nn = n * n
    print(f"bfs.{(x, y) = }")
    start = (x, y)
    dists = [[-1] * n for _ in range(n)]
    dists[y][x] = 0
    queue = deque([(x, y)])
    for (x, y), fish in fishes.items():
        if fish > size:
            dists[y][x] = nn
    for i in sea:
        print(*map(lambda x: "%2s" % x, i))
    dist = nn
    targets = deque([])
    while queue:
        x, y = queue.popleft()
        for dx, dy in ((1, 0), (-1, 0), (0, 1), (0, -1)):
            nx, ny = x + dx, y + dy
            next_dist = dists[y][x] + 1
            if 0 <= nx < n and 0 <= ny < n and dists[ny][nx] == -1:
                if (cell := sea[ny][nx]) <= size:
                    dists[ny][nx] = next_dist
                if 0 < cell < size:
                    if dist > dists[ny][nx]:
                        dist = dists[ny][nx]
                        targets = deque([(nx, ny)])
                    elif dist == dists[ny][nx]:
                        insort(targets, (nx, ny), key=lambda x: x[::-1])
                if next_dist <= dist:
                    queue.append((nx, ny))
    print()
    for i in dists:
        print(*map(lambda x: "%2s" % x, i))
    print(f"\n{start} -({dist})-> {targets[0]}" if targets else "end")
    return (*targets.popleft(), dist) if targets else False

In [92]:
def solution(open=open):
    inputs = iter(open(0).read().rstrip().split("\n"))
    n = int(next(inputs))
    sea = [list(map(int, input.split())) for input in inputs]

    secs = 0
    size = 2
    fishes = {}
    stomach = 0

    for y, line in enumerate(sea):
        for x, cell in enumerate(line):
            if 0 < cell < 9:
                fishes[x, y] = cell
            elif cell == 9:
                start = (x, y)
                sea[y][x] = 0

    x, y = start
    while fishes:
        target = bfs(x, y, size, n, fishes, sea)
        print(f"{target = }")
        if not target:
            break
        x, y, sec = target
        secs += sec
        fishes.pop((x, y))
        stomach += 1
        if stomach == size:
            print(f"size up! {size} -> {size + 1}")
            stomach = 0
            size += 1
        else:
            print(f"stomach/size = {stomach}/{size}")
        sea[y][x] = 0
        print()

    print(secs)

## 예제

In [93]:
# 백준 문제 풀이용 예제 실행 코드
from bwj import test

test_solution = test(solution)

# test_solution("""""")
# test_solution(read("fn").read())

In [94]:
test_solution(
    """6
5 4 3 2 3 4
4 3 2 3 4 5
3 2 9 5 6 6
2 1 2 3 4 5
3 2 1 6 5 4
6 6 6 6 6 6"""
)  # 60

bfs.(x, y) = (2, 2)
 5  4  3  2  3  4
 4  3  2  3  4  5
 3  2  0  5  6  6
 2  1  2  3  4  5
 3  2  1  6  5  4
 6  6  6  6  6  6

36 36 36 -1 36 36
36 36  1 36 36 36
36  1  0 36 36 36
 3  2  1 36 36 36
36  3  2 36 36 36
36 36 36 36 36 36

(2, 2) -(2)-> (1, 3)
target = (1, 3, 2)
stomach/size = 1/2

bfs.(x, y) = (1, 3)
 5  4  3  2  3  4
 4  3  2  3  4  5
 3  2  0  5  6  6
 2  0  2  3  4  5
 3  2  1  6  5  4
 6  6  6  6  6  6

36 36 36 -1 36 36
36 36  3 36 36 36
36  1  2 36 36 36
 1  0  1 36 36 36
36  1  2 36 36 36
36 36 36 36 36 36

(1, 3) -(2)-> (2, 4)
target = (2, 4, 2)
size up! 2 -> 3

bfs.(x, y) = (2, 4)
 5  4  3  2  3  4
 4  3  2  3  4  5
 3  2  0  5  6  6
 2  0  2  3  4  5
 3  2  0  6  5  4
 6  6  6  6  6  6

36 36 -1 -1 -1 36
36 -1 -1 -1 36 36
-1 -1  2 36 36 36
-1  2  1  2 36 36
 2  1  0 36 36 36
36 36 36 36 36 36

(2, 4) -(1)-> (2, 3)
target = (2, 3, 1)
stomach/size = 1/3

bfs.(x, y) = (2, 3)
 5  4  3  2  3  4
 4  3  2  3  4  5
 3  2  0  5  6  6
 2  0  0  3  4  5
 3  2  0  6  5  4

In [95]:
test_solution(
    """4
4 3 2 1
0 0 0 0
0 0 9 0
1 2 3 4"""
)  # 14

bfs.(x, y) = (2, 2)
 4  3  2  1
 0  0  0  0
 0  0  0  0
 1  2  3  4

16 16  2  3
 3  2  1  2
 2  1  0  1
 3  2 16 16

(2, 2) -(3)-> (3, 0)
target = (3, 0, 3)
stomach/size = 1/2

bfs.(x, y) = (3, 0)
 4  3  2  0
 0  0  0  0
 0  0  0  0
 1  2  3  4

16 16  1  0
 4  3  2  1
 5  4  3  2
 6  5 16 16

(3, 0) -(6)-> (0, 3)
target = (0, 3, 6)
size up! 2 -> 3

bfs.(x, y) = (0, 3)
 4  3  2  0
 0  0  0  0
 0  0  0  0
 0  2  3  4

16 -1 -1 -1
 2 -1 -1 -1
 1  2 -1 -1
 0  1  2 16

(0, 3) -(1)-> (1, 3)
target = (1, 3, 1)
stomach/size = 1/3

bfs.(x, y) = (1, 3)
 4  3  2  0
 0  0  0  0
 0  0  0  0
 0  0  3  4

16  3  4  5
 3  2  3  4
 2  1  2  3
 1  0  1 16

(1, 3) -(4)-> (2, 0)
target = (2, 0, 4)
stomach/size = 2/3

bfs.(x, y) = (2, 0)
 4  3  0  0
 0  0  0  0
 0  0  0  0
 0  0  3  4

16  1  0  1
 3  2  1  2
 4  3  2  3
 5  4  3 16
end
target = False
14


In [96]:
test_solution(
    """3
0 0 0
0 0 0
0 9 0"""
)  # 0

0


In [97]:
test_solution(
    """3
0 0 1
0 0 0
0 9 0"""
)  # 3

bfs.(x, y) = (1, 2)
 0  0  1
 0  0  0
 0  0  0

 3  2  3
 2  1  2
 1  0  1

(1, 2) -(3)-> (2, 0)
target = (2, 0, 3)
stomach/size = 1/2

3


In [98]:
test_solution(
    """6
6 0 6 0 6 1
0 0 0 0 0 2
2 3 4 5 6 6
0 0 0 0 0 2
0 2 0 0 0 0
3 9 3 0 0 1"""
)  # 48

bfs.(x, y) = (1, 5)
 6  0  6  0  6  1
 0  0  0  0  0  2
 2  3  4  5  6  6
 0  0  0  0  0  2
 0  2  0  0  0  0
 3  0  3  0  0  1

36  7 36 -1 36 -1
 5  6  7 -1 -1 -1
 4 36 36 36 36 36
 3  2  3  4  5  6
 2  1  2  3  4  5
36  0 36  4  5  6

(1, 5) -(6)-> (5, 5)
target = (5, 5, 6)
stomach/size = 1/2

bfs.(x, y) = (5, 5)
 6  0  6  0  6  1
 0  0  0  0  0  2
 2  3  4  5  6  6
 0  0  0  0  0  2
 0  2  0  0  0  0
 3  0  3  0  0  0

36 11 36 13 36 15
 9 10 11 12 13 14
 8 36 36 36 36 36
 7  6  5  4  3  2
 6  5  4  3  2  1
36  6 36  2  1  0

(5, 5) -(15)-> (5, 0)
target = (5, 0, 15)
size up! 2 -> 3

bfs.(x, y) = (5, 0)
 6  0  6  0  6  0
 0  0  0  0  0  2
 2  3  4  5  6  6
 0  0  0  0  0  2
 0  2  0  0  0  0
 3  0  3  0  0  0

36 -1 36 -1 36  0
-1 -1 -1 -1  2  1
-1 -1 36 36 36 36
-1 -1 -1 -1 -1 -1
-1 -1 -1 -1 -1 -1
-1 -1 -1 -1 -1 -1

(5, 0) -(1)-> (5, 1)
target = (5, 1, 1)
stomach/size = 1/3

bfs.(x, y) = (5, 1)
 6  0  6  0  6  0
 0  0  0  0  0  0
 2  3  4  5  6  6
 0  0  0  0  0  2
 0  2  0  0  0 

In [99]:
test_solution(
    """6
1 1 1 1 1 1
2 2 6 2 2 3
2 2 5 2 2 3
2 2 2 4 6 3
0 0 0 0 0 6
0 0 0 0 0 9"""
)  # 39

bfs.(x, y) = (5, 5)
 1  1  1  1  1  1
 2  2  6  2  2  3
 2  2  5  2  2  3
 2  2  2  4  6  3
 0  0  0  0  0  6
 0  0  0  0  0  0

10  9 10 -1 -1 -1
 9  8 36 -1 -1 36
 8  7 36 -1 -1 36
 7  6  5 36 36 36
 6  5  4  3  2 36
 5  4  3  2  1  0

(5, 5) -(9)-> (1, 0)
target = (1, 0, 9)
stomach/size = 1/2

bfs.(x, y) = (1, 0)
 1  0  1  1  1  1
 2  2  6  2  2  3
 2  2  5  2  2  3
 2  2  2  4  6  3
 0  0  0  0  0  6
 0  0  0  0  0  0

 1  0  1  2 -1 -1
 2  1 36 -1 -1 36
-1  2 36 -1 -1 36
-1 -1 -1 36 36 36
-1 -1 -1 -1 -1 36
-1 -1 -1 -1 -1 -1

(1, 0) -(1)-> (0, 0)
target = (0, 0, 1)
size up! 2 -> 3

bfs.(x, y) = (0, 0)
 0  0  1  1  1  1
 2  2  6  2  2  3
 2  2  5  2  2  3
 2  2  2  4  6  3
 0  0  0  0  0  6
 0  0  0  0  0  0

 0  1  2 -1 -1 -1
 1  2 36 -1 -1 -1
 2 -1 36 -1 -1 -1
-1 -1 -1 36 36 -1
-1 -1 -1 -1 -1 36
-1 -1 -1 -1 -1 -1

(0, 0) -(1)-> (0, 1)
target = (0, 1, 1)
stomach/size = 1/3

bfs.(x, y) = (0, 1)
 0  0  1  1  1  1
 0  2  6  2  2  3
 2  2  5  2  2  3
 2  2  2  4  6  3
 0  0  0  0  0  6